In [3]:
import numpy as np

def generate_custom_dataset():
    # Definir o número de amostras por classe
    n_samples_per_class = [8, 8, 7]

    # Definir o número total de classes
    n_classes = len(n_samples_per_class)

    # Definir o número de atributos por amostra
    n_features = 2

    # Inicializar uma lista para armazenar as amostras e rótulos
    X = []
    y = []

    # Gerar amostras para cada classe
    for class_idx, n_samples in enumerate(n_samples_per_class):
        # Gerar amostras com base na classe
        for _ in range(n_samples):
            # Amostra com distribuição normal centrada em (class_idx, class_idx)
            sample = np.random.normal(loc=class_idx, scale=0.5, size=n_features)
            X.append(sample)
            y.append(class_idx)

    # Converter listas para arrays numpy
    X = np.array(X)
    y = np.array(y)

    return X, y

# Teste da função
X_custom, y_custom = generate_custom_dataset()
print("Dataset Customizado:")
print("X:")
print(X_custom)
print("y:")
print(y_custom)
print("Forma de X:", X_custom.shape)
print("Forma de y:", y_custom.shape)


Dataset Customizado:
X:
[[ 0.29254831  0.33544821]
 [ 0.24928645 -0.72765086]
 [ 0.42051987  0.36378828]
 [ 0.74722536 -0.32816969]
 [ 0.40030942  1.13913346]
 [-0.01563448  0.66438469]
 [ 0.8043956  -1.22610012]
 [-0.10832735 -0.51920298]
 [ 0.43018038  1.31389117]
 [ 0.48007803  0.42317186]
 [ 1.99759041  1.48507023]
 [ 0.59285089  0.50127209]
 [ 0.58177213  1.17562229]
 [ 1.42983019  1.52786962]
 [ 1.05014746  0.55807771]
 [ 1.12760409  0.95100691]
 [ 1.54954316  1.89834662]
 [ 0.65315017  1.90607769]
 [ 1.51344862  2.31750303]
 [ 1.94863639  1.9087139 ]
 [ 2.79820297  0.87661481]
 [ 1.75810875  0.3664212 ]
 [ 1.80015595  1.79817396]]
y:
[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2]
Forma de X: (23, 2)
Forma de y: (23,)
